In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
train_path = '../input/chest-xray-pneumonia/chest_xray/train/'
val_path = '../input/chest-xray-pneumonia/chest_xray/val/'
test_path = '../input/chest-xray-pneumonia/chest_xray/test/'

In [3]:
# re-size all the images to a size VGG-16 expects.
IMAGE_SIZE = [224, 224]

# Set the batch size
BATCH_SIZE = 32  # try reducing batch size or freeze more layers if your GPU runs out of memory
NUM_EPOCHS = 5
LEARNING_RATE = 0.0001
NUM_CLASSES = 2 # We are aware of it.

In [4]:
import os
CLASSES = os.listdir(train_path)
NUM_CLASSES = len(CLASSES)

In [5]:
print(NUM_CLASSES)

2


In [6]:
print("Class --> {} \n and the length is : {}".format(CLASSES, NUM_CLASSES))


Class --> ['PNEUMONIA', 'NORMAL'] 
 and the length is : 2


In [7]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [8]:
training_set = train_datagen.flow_from_directory(
    directory = train_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 5216 images belonging to 2 classes.


In [9]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
test_set = test_datagen.flow_from_directory(
    directory = test_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 624 images belonging to 2 classes.


In [11]:
# Import the VGG 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [12]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [13]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

prediction = Dense(NUM_CLASSES, activation='softmax')(x)

In [14]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [19]:
# callbacks
import tensorflow as tf
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath="model.h5", save_best_only=True)
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(patience=3)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=10)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./pneumonia_clf_vgg16_logs")

callbacks = [checkpoint_callback, reduce_lr_callback, early_stopping_callback,  tensorboard_callback ]

In [21]:
history = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
    callbacks = callbacks
)

Epoch 1/5
163/163 [==============================] - 176s 1s/step - loss: 0.1467 - accuracy: 0.9425 - val_loss: 0.3554 - val_accuracy: 0.8766 - lr: 0.0010
Epoch 2/5
163/163 [==============================] - 133s 817ms/step - loss: 0.1614 - accuracy: 0.9417 - val_loss: 0.3958 - val_accuracy: 0.8878 - lr: 0.0010
Epoch 3/5
163/163 [==============================] - 133s 816ms/step - loss: 0.1009 - accuracy: 0.9617 - val_loss: 0.3467 - val_accuracy: 0.9022 - lr: 0.0010
Epoch 4/5
163/163 [==============================] - 133s 819ms/step - loss: 0.0877 - accuracy: 0.9674 - val_loss: 0.3971 - val_accuracy: 0.9071 - lr: 0.0010
Epoch 5/5
163/163 [==============================] - 134s 819ms/step - loss: 0.0807 - accuracy: 0.9701 - val_loss: 0.2645 - val_accuracy: 0.9199 - lr: 0.0010


In [22]:
model.evaluate(test_set)

20/20 [==============================] - 6s 318ms/step - loss: 0.2645 - accuracy: 0.9199


[0.26449674367904663, 0.9198718070983887]

In [23]:
model.save( "pneumonia_clf_vgg16.h5" )

In [24]:
import tensorflow as tf
loaded = tf.keras.models.load_model("pneumonia_clf_vgg16.h5")

In [25]:
loaded == model

False

In [26]:
loaded.evaluate(test_set)

20/20 [==============================] - 7s 315ms/step - loss: 0.2645 - accuracy: 0.9199


[0.26449674367904663, 0.9198718070983887]

In [27]:
# callbacks
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath="model.h5", save_best_only=True)
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(patience=3)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=10)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")